This notebook is a utility to move and organize files for the block based synchronization paradigm - it is a pre-requisite for running the pipeline

In [22]:
import pathlib
import os
import shutil
import datetime
import subprocess as sp
import re
import pandas as pd
import numpy as np
import tqdm
import glob
from lxml import etree
from BlockSync2 import *
import scipy.io

In [ ]:
def get_paths_to_record_nodes(tree_root_path, export_path=False):
    """
    This function looks for record node folders and extracts them to a .mat file that can be used with the Generate_Metadata_with_timeSeriesViewer.mlx file located at Nimrod's desktop on the MSI server
    :param tree_root_path:
    :return:
    """
    # define some helper functions
    def match_date_pattern(string):
        pattern = r'^\d{4}_\d{2}_\d{2}$'
        return bool(re.match(pattern, string))

    def match_block_pattern(string):
        pattern = r'^block_\d{3}$'
        return bool(re.match(pattern, string))

    def convert_paths_to_unix(paths, relative_to):
        unix_paths = []
        for path in paths:
            path = path.relative_to(relative_to)
            unix_path = pathlib.Path(path).as_posix()
            unix_paths.append(unix_path)
        return unix_paths

    path_list = [] # initialize a variable to collect the paths into
    date_folder_list = [i for i in tree_root_path.iterdir() if match_date_pattern(str(i.name)) and i.is_dir()] # use function to find date folder
    for date_folder in date_folder_list: # iterate over date folders
        for block in [b for b in date_folder.iterdir() if match_block_pattern(str(b.name)) and b.is_dir()]: # use function to find block folders
            oe_path = block / 'oe_files' # enter the oe_files folder
            datetime_folder_name = [i.name for i in oe_path.iterdir()][0]
            datetime_path = oe_path / datetime_folder_name
            rec_node_name = [i.name for i in datetime_path.iterdir() if 'Node' in (str(i.name))][0]
            rec_node_path = datetime_path / rec_node_name
            path_list.append(rec_node_path)



In [18]:
import re

def match_date_pattern(string):
    pattern = r'^\d{4}_\d{2}_\d{2}$'
    return bool(re.match(pattern, string))

def match_block_pattern(string):
    pattern = r'^block_\d{3}$'
    return bool(re.match(pattern, string))

def convert_paths_to_unix(paths, relative_to):
    unix_paths = []
    for path in paths:
        path = path.relative_to(relative_to)
        unix_path = pathlib.Path(path).as_posix()
        unix_paths.append(unix_path)
    return unix_paths

path_list = []
tree_root_path = pathlib.Path('Z:/Nimrod/experiments/PV_62')
date_folder_list = [i for i in tree_root_path.iterdir() if match_date_pattern(str(i.name)) and i.is_dir()]
for date_folder in date_folder_list:
    for block in [b for b in date_folder.iterdir() if match_block_pattern(str(b.name)) and b.is_dir()]:
        oe_path = block / 'oe_files'
        datetime_folder_name = [i.name for i in oe_path.iterdir()][0]
        datetime_path = oe_path / datetime_folder_name
        rec_node_name = [i.name for i in datetime_path.iterdir() if 'Node' in (str(i.name))][0]
        rec_node_path = datetime_path / rec_node_name
        path_list.append(rec_node_path)

relative_to = pathlib.Path('Z:/Nimrod/experiments')
unix_paths = convert_paths_to_unix(path_list,relative_to=relative_to)
export_path = pathlib.Path(r'Z:\Nimrod\HelperFiles\record_node_locations_pv62.mat')
mat_format = {'data': unix_paths}
scipy.io.savemat(export_path, mat_format)

[WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_22/block_005/oe_files/2023-04-22_14-30-15/Record Node 108'), WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_22/block_006/oe_files/2023-04-22_14-34-56/Record Node 108'), WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_22/block_007/oe_files/2023-04-22_14-41-49/Record Node 108'), WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_22/block_008/oe_files/2023-04-22_14-57-30/Record Node 108'), WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_22/block_009/oe_files/2023-04-22_15-14-56/Record Node 108'), WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_24/block_010/oe_files/2023-04-24_16-49-28/Record Node 108'), WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_24/block_011/oe_files/2023-04-24_17-09-35/Record Node 108'), WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_24/block_012/oe_files/2023-04-24_17-27-10/Record Node 108'), WindowsPath('Z:/Nimrod/experiments/PV_62/2023_04_24/block_013/oe_files/2023-04-24_17-47-56/Record Node 108'), WindowsPa

In [23]:
def convert_paths_to_unix(paths, relative_to):
    unix_paths = []
    for path in paths:
        path = path.relative_to(relative_to)
        unix_path = pathlib.Path(path).as_posix()
        unix_paths.append(unix_path)
    return unix_paths
relative_to = pathlib.Path('Z:/Nimrod/experiments')
unix_paths = convert_paths_to_unix(path_list,relative_to=relative_to)

export_path = pathlib.Path(r'Z:\Nimrod\HelperFiles\record_node_locations_pv62.mat')
mat_format = {'data': unix_paths}
scipy.io.savemat(export_path, mat_format)

In [11]:
# This code converts audio files in a given folder from m4a to mp3:
folder_path = pathlib.Path(r'D:/Marks3/Music/RawAudio')
input_format = 'm4a'
files_in_folder = [i for i in folder_path.iterdir()]
files_to_convert = [i for i in files_in_folder if str(input_format) in i.name]
for f in files_to_convert:
    print(f'converting {f} to mp3')
    sp.run(f'ffmpeg -i {f} -c:a libmp3lame -q:a 2 {str(f)[:-3] + "mp3"}')
    print('done')

converting D:\Marks3\Music\RawAudio\505174__kbriber__lizard-eating.m4a to mp3


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [15]:
folder_path = pathlib.Path(r'D:/Marks3/Music/RawAudio')
input_format = 'm4a'
files_in_folder = [i for i in folder_path.iterdir()]
files_to_convert = [i for i in files_in_folder if str(input_format) in i.name]
for f in files_to_convert:
    yn = input(f' will try converting {f} to mp3 - proceed?')
    if yn == 'y':
        print(f'ffmpeg -i "{str(f)} -c:a libmp3lame -q:a 2 {str(f)[:-3] + "mp3"}')
    else:
        print('not converting, next')
print('done')

not converting, next
not converting, next
ffmpeg -i D:\Marks3\Music\RawAudio\Door inside.m4a -c:a libmp3lame -q:a 2 D:\Marks3\Music\RawAudio\Door inside.mp3
ffmpeg -i D:\Marks3\Music\RawAudio\Door outside.m4a -c:a libmp3lame -q:a 2 D:\Marks3\Music\RawAudio\Door outside.mp3
ffmpeg -i D:\Marks3\Music\RawAudio\Feedersound.m4a -c:a libmp3lame -q:a 2 D:\Marks3\Music\RawAudio\Feedersound.mp3
ffmpeg -i D:\Marks3\Music\RawAudio\Natural sounds _ glass door.m4a -c:a libmp3lame -q:a 2 D:\Marks3\Music\RawAudio\Natural sounds _ glass door.mp3
done


In [9]:
str(files_to_convert[0])[:-3] + "mp3"

'D:\\Marks3\\Music\\RawAudio\\505174__kbriber__lizard-eating.mp3'

In [100]:
def move_dlc_analysis_files(directory_path, substring='DLC'):
    directory_path = pathlib.Path(directory_path)
    files = directory_path.rglob('*')
    new_dir = directory_path / "prev_analysis"
    try:
        new_dir.mkdir()
        print(f'made {new_dir}')
    except FileExistsError:
        print('prev_analysis already exists')

    for file in files:
        if file.is_file() and substring in file.name:
            new_path = new_dir / file.name
            print('file is ', file)
            print('moving to', new_path)
            shutil.move(file,new_path)
            print(f'moved {file.name} to {new_path}')






In [101]:
# search and destroy dlc files from previous annotation iterations:
p = pathlib.Path(rf'Z:\Nimrod\experiments\PV_62')
date_folder_list = [i for i in p.iterdir() if 'unsorted' not in str(i).lower() and i.is_dir()]
for date_folder in date_folder_list:
    # list all blocks in the folder:
    block_list = [i for i in date_folder.iterdir()]
    for block in block_list:
        directory_LE = block / 'eye_videos' / 'LE'
        print([i for i in directory_LE.iterdir()][0])
        internal_dir = directory_LE / [i for i in directory_LE.iterdir()][0]
        move_dlc_analysis_files(directory_path=internal_dir)
        directory_RE = block / 'eye_videos' / 'RE'
        print([i for i in directory_RE.iterdir()][0])
        internal_dir = directory_RE / [i for i in directory_RE.iterdir()][0]
        move_dlc_analysis_files(directory_path=internal_dir)




Z:\Nimrod\experiments\PV_62\2023_04_22\block_005\eye_videos\LE\pv62_exp5_bugs_640x480_60hz_experiment_1_recording_0
prev_analysis already exists
file is  Z:\Nimrod\experiments\PV_62\2023_04_22\block_005\eye_videos\LE\pv62_exp5_bugs_640x480_60hz_experiment_1_recording_0\prev_analysis\pv62_exp5_bugs_LEDLC_resnet_50_Eye_Tracking_piplineMar1shuffle1_950000.csv
moving to Z:\Nimrod\experiments\PV_62\2023_04_22\block_005\eye_videos\LE\pv62_exp5_bugs_640x480_60hz_experiment_1_recording_0\prev_analysis\pv62_exp5_bugs_LEDLC_resnet_50_Eye_Tracking_piplineMar1shuffle1_950000.csv
moved pv62_exp5_bugs_LEDLC_resnet_50_Eye_Tracking_piplineMar1shuffle1_950000.csv to Z:\Nimrod\experiments\PV_62\2023_04_22\block_005\eye_videos\LE\pv62_exp5_bugs_640x480_60hz_experiment_1_recording_0\prev_analysis\pv62_exp5_bugs_LEDLC_resnet_50_Eye_Tracking_piplineMar1shuffle1_950000.csv
file is  Z:\Nimrod\experiments\PV_62\2023_04_22\block_005\eye_videos\LE\pv62_exp5_bugs_640x480_60hz_experiment_1_recording_0\prev_analysi

In [99]:
#clean prev_analysis folders in wrong place
p = pathlib.Path(rf'Z:\Nimrod\experiments\PV_62')
dest = pathlib.Path(rf'Z:\Nimrod\experiments\PV_62\UnSorted\errors')
date_folder_list = [i for i in p.iterdir() if 'unsorted' not in str(i).lower() and i.is_dir()]
for date_folder in date_folder_list:
    # list all blocks in the folder:
    block_list = [i for i in date_folder.iterdir()]
    for block in block_list:
        directory_LE = block / 'eye_videos' / 'LE'
        folders = [i for i in directory_LE.glob('*')]
        for folder in folders:
            if folder.name == 'prev_analysis':
                eye = 'LE'
                # print(f'would have moved {folder} to {dest / block.name / eye / folder.name}')
                shutil.move(folder, dest / block.name / eye / folder.name)
        directory_RE = block / 'eye_videos' / 'RE'
        folders = [i for i in directory_RE.glob('*')]
        for folder in folders:
            if folder.name == 'prev_analysis':
                eye = 'RE'
                # print(f'would have moved {folder} to {dest / block.name / eye / folder.name}')
                shutil.move(folder, dest / block.name / eye / folder.name)



In [104]:
# clean empty prev_analysis folders:
def is_directory_empty(directory_path):
    with os.scandir(directory_path) as entries:
        for entry in entries:
            if entry.is_file() or entry.is_dir():
                return False
    return True

p = pathlib.Path(rf'Z:\Nimrod\experiments\PV_62')
dest = pathlib.Path(rf'Z:\Nimrod\experiments\PV_62\UnSorted\errors')
date_folder_list = [i for i in p.iterdir() if 'unsorted' not in str(i).lower() and i.is_dir()]
for date_folder in date_folder_list:
    # list all blocks in the folder:
    block_list = [i for i in date_folder.iterdir()]
    for block in block_list:
        directory_LE = block / 'eye_videos' / 'LE'
        internal_dir = [i for i in directory_LE.glob('*')][0]
        folders = [i for i in internal_dir.glob('*')]
        for folder in folders:
            if folder.name == 'prev_analysis' and is_directory_empty(folder):
                eye = 'LE'
                #print(f'would have moved {folder} to {dest / block.name / eye / folder.name}')
                shutil.move(folder, dest / block.name / eye / folder.name)
        directory_RE = block / 'eye_videos' / 'RE'
        internal_dir = [i for i in directory_RE.glob('*')][0]
        folders = [i for i in internal_dir.glob('*')]
        for folder in folders:
            if folder.name == 'prev_analysis' and is_directory_empty(folder):
                eye = 'RE'
                #print(f'would have moved {folder} to {dest / block.name / eye / folder.name}')
                shutil.move(folder, dest / block.name / eye / folder.name)

AttributeError: 'WindowsPath' object has no attribute 'rstrip'

In [62]:
# correct nested prev_analysis folder:
p = pathlib.Path(rf'Z:\Nimrod\experiments\PV_62')
date_folder_list = [i for i in p.iterdir() if 'unsorted' not in str(i).lower() and i.is_dir()]
directory = date_folder_list[0] / 'block_005'/ 'eye_videos' / 'LE' / 'pv62_exp5_bugs_640x480_60hz_experiment_1_recording_0'
files = directory.rglob('*/')
for file in files:
    if 'DLC' in str(file):
        shutil.move(file,directory / file.name)


In [4]:
## This section creates a .csv based report on all experimental blocks for a single animal
df_dict = {
    'date':[],
    'block':[],
    'oe_file':[],
    'eye_vids_h264':[],
    'eye_vids_mp4':[],
    'eye_vids_labeled':[],
    'arena_vids':[],
    'notes':[]
}
df = pd.DataFrame(df_dict)

p = pathlib.Path(rf'Z:\Nimrod\experiments\PV_62')
date_folder_list = [i for i in p.iterdir() if 'unsorted' not in str(i).lower() and i.is_dir()]
for date_folder in date_folder_list:
    # list all blocks in the folder:
    block_list = [i for i in date_folder.iterdir()]
    for block in block_list:
        analysis = [i for i in (block / 'analysis').iterdir()]
        h264s = [ pathlib.Path(f).name for f in glob.glob(str(block / 'eye_videos') + r'\**\*.h264',recursive=True) if 'DLC' not in str(f)]
        mp4s = [ pathlib.Path(f).name for f in glob.glob(str(block / 'eye_videos') + r'\**\*.mp4',recursive=True) if 'DLC' not in str(f)]
        labeled_videos = [ pathlib.Path(f).name for f in glob.glob(str(block / 'eye_videos') + r'\**\*.mp4',recursive=True) if 'DLC' in str(f)]
        arena_videos = [ pathlib.Path(f).name for f in glob.glob(str(block / 'arena_videos') + r'\**\*.mp4',recursive=True)]
        try:
            oe_file = [i for i in (block / 'oe_files').iterdir()][0].name
        except IndexError:
            oe_file = None
        df_row = {
        'date':date_folder.name,
        'block':block.name[6:],
        'eye_vids_h264':h264s,
        'eye_vids_mp4':mp4s,
        'eye_vids_labeled':labeled_videos,
        'arena_vids':arena_videos,
        'oe_file': oe_file
        }
        i = len(df)
        df.loc[i] = df_row




c:\users\marks3\pycharmprojects\pythonproject3\venv\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [5]:
df

,date,block,oe_file,eye_vids_h264,eye_vids_mp4,eye_vids_labeled,arena_vids,notes
0,2023_04_22,005,2023-04-22_14-30-15,"[pv62_exp5_bugs.h264, pv62_exp5_bugs.h264]","[pv62_exp5_bugs_LE.mp4, pv62_exp5_bugs.mp4]",[pv62_exp5_bugs_LEDLC_resnet_50_Eye_Tracking_p...,[],NaN
1,2023_04_22,006,2023-04-22_14-34-56,"[pv62_exp6_bugs.h264, pv62_exp6_bugs.h264]","[pv62_exp6_bugs_LE.mp4, pv62_exp6_bugs.mp4]",[pv62_exp6_bugs_LEDLC_resnet50_pogona_eye_trac...,"[back_20230422T143506.mp4, left_20230422T14350...",NaN
2,2023_04_22,007,2023-04-22_14-41-49,"[pv62_exp7_bugs.h264, pv62_exp7_bugs.h264]","[pv62_exp7_bugs_LE.mp4, pv62_exp7_bugs.mp4]",[pv62_exp7_bugs_LEDLC_resnet50_pogona_eye_trac...,"[back_20230422T144207.mp4, back_20230422T14520...",NaN
3,2023_04_22,008,2023-04-22_14-57-30,"[pv62_exp8_bugs.h264, pv62_exp8_bugs.h264]","[pv62_exp8_bugs_LE.mp4, pv62_exp8_bugs.mp4]",[pv62_exp8_bugs_LEDLC_resnet50_pogona_eye_trac...,"[back_20230422T145741.mp4, back_20230422T15074...",NaN
4,2023_04_22,009,2023-04-22_15-14-56,"[pv62_exp9_bugs.h264, pv62_exp9_bugs.h264]","[pv62_exp9_bugs_LE.mp4, pv62_exp9_bugs.mp4]",[pv62_exp9_bugs_LEDLC_resnet50_pogona_eye_trac...,[],NaN
...,...,...,...,...,...,...,...,...
61,2023_05_11,066,2023-05-11_14-08-04,"[230511_pv62_trial4.h264, 230511_pv62_trial4.h...","[230511_pv62_trial4_LE.mp4, 230511_pv62_trial4...",[],[],NaN
62,2023_05_18,067,2023-05-18_11-26-48,"[230518_pv62_trial1.h264, 230518_pv62_trial1.h...","[230518_pv62_trial1_LE.mp4, 230518_pv62_trial1...",[],[],NaN
63,2023_05_18,068,2023-05-18_12-06-08,"[230518_pv62_trial2.h264, 230518_pv62_trial2.h...","[230518_pv62_trial2_LE.mp4, 230518_pv62_trial2...",[],[],NaN
64,2023_05_18,069,2023-05-18_13-08-27,"[230518_pv62_trial3.h264, 230518_pv62_trial3.h...","[230518_pv62_trial3_LE.mp4, 230518_pv62_trial3...",[],[],NaN


In [28]:
## This cell re-organizes date names
# find all folders to rename:
p = pathlib.Path(r'Z:\Nimrod\experiments\PV_24')
date_folder_list = [i for i in p.iterdir() if 'block' not in str(i).lower() and i.is_dir()]
for i in date_folder_list:
    date = i.name
    day = date[0:2]
    month = date[3:5]
    year = date[-4:]
    new_name = year+'_'+month+'_'+day
    new_path = i.parent / new_name
    i.rename(new_path)



In [2]:
p = pathlib.Path('Z:')
p.exists()

True

In [4]:
# This cell organizes block names into block_XXX format
# iterate over all date folders:
p = pathlib.Path(rf'Z:\Nimrod\experiments\PV_24')
date_folder_list = [i for i in p.iterdir() if 'block' not in str(i).lower() and i.is_dir()]
for date_folder in date_folder_list:
    # list all blocks in the folder:
    folder_list = [i for i in date_folder.iterdir()]
    for block in folder_list:
        # find the block number
        name = block.name
        # There should be a regex thing here to deal with different possible locations
        pattern = re.compile(r"\d+")
        matches = pattern.finditer(name)
        # correct the number to the new format
        for match in matches:
            num = match.group()
            span = match.span()
        if len(num) == 1:
            new_num = f'00{num}'
        elif len(num) == 2:
            new_num = f'0{num}'
        else:
            new_num = num
        #print(f'the number {num} became {new_num}')
        new_block_name = f'block_{new_num}'
        new_block_path = block.parent / new_block_name
        #print(f'would have renamed {block} to {new_block_path}')
        block.rename(new_block_path)

In [38]:
block.parent / new_block_name

WindowsPath('Z:/Nimrod/experiments/PV_24/2022_03_13/block_096')

In [114]:
# Rename AUX labeled files as CHxs
p = pathlib.Path(r'Z:\Nimrod\experiments\PV_24')
date_folder_list = [i for i in p.iterdir() if 'BAD_BLOCK' not in str(i) and (p/str(i)).is_dir() and 'template' not in str(i)]
for date_folder in date_folder_list:
    for block in (p / date_folder).iterdir():
        oe_folder = block / 'oe_files'
        oe_folder_name = [i for i in oe_folder.iterdir() if i.is_dir]
        try:
            oe_folder = oe_folder_name[0] / r'Record Node 101'
        except IndexError:
            print('no open ephys folder for', block)
        for i in oe_folder.iterdir():
            if 'AUX' in str(i):
                ch_num = str(i)[-12]
                ch_num = str(int(ch_num) + 32)
                destination = i.parent / (str(i.name[:4]) + 'CH' + ch_num + '.continuous')
                #os.rename(i,destination)

no open ephys folder for Z:\Nimrod\experiments\PV_24\04_01_2022\block_65_BAD_BLOCK


In [113]:
# extract hidehere files back into the block:
p = pathlib.Path(r'Z:\Nimrod\experiments\PV_24')
date_folder_list = [i for i in p.iterdir() if 'BAD_BLOCK' not in str(i) and (p/str(i)).is_dir() and 'template' not in str(i)]
for date_folder in date_folder_list:
    for block in (p / date_folder).iterdir():
        oe_folder = block / 'oe_files'
        oe_folder_name = [i for i in oe_folder.iterdir() if i.is_dir]
        try:
            oe_folder = oe_folder_name[0] / r'Record Node 101'
        except IndexError:
            print('no open ephys folder for', block)
        if (oe_folder / 'hidehere').is_dir():
            print(f'take care of block {oe_folder}')
        else:
            print(f'nothing here')


nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
no open ephys folder for Z:\Nimrod\experiments\PV_24\04_01_2022\block_65_BAD_BLOCK
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing here
nothing

In [ ]:
def produce_analysis_reportV2(blocklist):



In [73]:
### CREATE A STATUS REPORT FOR ALL BLOCKS ###
def block_generator(block_list,experiment_path,animal, bad_blocks=[]):
    """
    creates a block_collection to iterate over with multi-block functions
    :param block_list: list of block numbers to loop over
    :param experiment_path: pathlib.Path instance to the experiment folder
    :param animal: string
    :param bad_blocks: blocks to ignore
    :return:
    """
    p = pathlib.Path(experiment_path) / animal
    date_folder_list = [i for i in p.iterdir() if 'block' not in str(i).lower() and i.is_dir()]
    block_collection = []
    for date_path in date_folder_list:
        date = date_path.name
        # list all the blocks in the folder:
        folder_list = [i for i in date_path.iterdir()]
        for block in folder_list:
            block_number = block.name[-3:]
            if int(block_number) in block_list and int(block_number) not in bad_blocks:
                #block definition
                block = BlockSync(animal_call=animal,
                                  experiment_date=date,block_num=block_number,
                                  path_to_animal_folder=str(experiment_path))
                block_collection.append(block)
    return block_collection

animal_folder = pathlib.Path(r'Z:\Nimrod\experiments')
animal = 'PV_24'
blocks_to_create = [i for i in range(0,97)]
bad_blocks = [59,79,80,73]
blocklist = block_generator(blocks_to_create,animal_folder,animal,bad_blocks=bad_blocks)
df_inds = [b.block_num for b in blocklist]


df = pd.DataFrame(data=None, index=df_inds, columns=['block_num','automated_step','manual_synchronization','bokeh_plot', 'saccade_analysis', 'accel_analysis', 'db_ratio'])

for i, block in enumerate(blocklist):
    ind = df.index[i]
    df.at[ind,'block_num'] = str(block.block_num)
    # collect it:
    a_files = os.listdir(block.block_path / 'analysis')
    if 'blocksync_df.csv' in a_files:
        df.at[ind,'automated_step'] = True
    if len([f for f in a_files if '.html' in f]) != 0:
        df.at[ind,'bokeh_plot'] = True
    if 'manual_sync_df.csv' in a_files:
        df.at[ind,'manual_synchronization'] = True
    if 'l_saccades.csv' in a_files:
        df.at[ind,'saccade_analysis'] = True
    # find the accel file location:
    if (block.oe_path / 'analysis').is_dir():
        oep = block.oe_path / 'analysis'
        oe_files = glob.glob(str(oep)+'\*\*',recursive=True)

        for f in oe_files:
            if 'lizMov' in f:
                df.at[ind,'accel_analysis'] = True
            elif 'dbRatio' in f:
                df.at[ind,'db_ratio'] = True

df.to_csv(r'Z:\Nimrod\experiments\PV_24_analysis_report.csv',index=False)


instantiated block number 001 at Path: Z:\Nimrod\experiments\PV_24\2021_12_23\block_001
Found the sample rate for block 001 in the xml file, it is 20000 Hz
instantiated block number 002 at Path: Z:\Nimrod\experiments\PV_24\2021_12_23\block_002
Found the sample rate for block 002 in the xml file, it is 20000 Hz
instantiated block number 003 at Path: Z:\Nimrod\experiments\PV_24\2021_12_23\block_003
Found the sample rate for block 003 in the xml file, it is 20000 Hz
instantiated block number 004 at Path: Z:\Nimrod\experiments\PV_24\2021_12_23\block_004
Found the sample rate for block 004 in the xml file, it is 20000 Hz
instantiated block number 005 at Path: Z:\Nimrod\experiments\PV_24\2021_12_23\block_005
Found the sample rate for block 005 in the xml file, it is 20000 Hz
instantiated block number 006 at Path: Z:\Nimrod\experiments\PV_24\2021_12_26\block_006
Found the sample rate for block 006 in the xml file, it is 20000 Hz
instantiated block number 007 at Path: Z:\Nimrod\experiments\PV_

In [72]:
block.block_num

'096'

In [65]:
df = pd.DataFrame(data=None, index=df_inds, columns=['automated_step','manual_synchronization','bokeh_plot', 'saccade_analysis', 'accel_analysis', 'db_ratio'])

for i, block in enumerate(blocklist):
    ind = df.index[i]
    # collect it:
    a_files = os.listdir(block.block_path / 'analysis')
    if 'blocksync_df.csv' in a_files:
        df.at[ind,'automated_step'] = True
    if len([f for f in a_files if '.html' in f]) != 0:
        df.at[ind,'bokeh_plot'] = True
    if 'manual_sync_df.csv' in a_files:
        df.at[ind,'manual_synchronization'] = True
    if 'l_saccades.csv' in a_files:
        df.at[ind,'saccade_analysis'] = True
    # find the accel file location:
    if (block.oe_path / 'analysis').is_dir():
        oep = block.oe_path / 'analysis'
        oe_files = glob.glob(str(oep)+'\*\*',recursive=True)

        for f in oe_files:
            if 'lizMov' in f:
                df.at[ind,'accel_analysis'] = True
            elif 'dbRatio' in f:
                df.at[ind,'db_ratio'] = True



In [66]:
df

,automated_step,manual_synchronization,bokeh_plot,saccade_analysis,accel_analysis,db_ratio
001,True,NaN,NaN,NaN,True,NaN
002,NaN,NaN,NaN,NaN,True,NaN
003,NaN,NaN,NaN,NaN,True,NaN
004,NaN,NaN,NaN,NaN,True,NaN
005,NaN,NaN,NaN,NaN,True,NaN
...,...,...,...,...,...,...
091,NaN,NaN,NaN,NaN,True,NaN
092,NaN,NaN,NaN,NaN,True,NaN
093,NaN,NaN,NaN,NaN,True,NaN
095,NaN,NaN,NaN,NaN,True,NaN


In [47]:
df

,automated_step,manual_synchronization,bokeh_plot,saccade_analysis,accel_analysis,db_ratio
001,True,NaN,NaN,NaN,NaN,NaN
002,NaN,NaN,NaN,NaN,NaN,NaN
003,NaN,NaN,NaN,NaN,NaN,NaN
004,NaN,NaN,NaN,NaN,NaN,NaN
005,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
091,NaN,NaN,NaN,NaN,NaN,NaN
092,NaN,NaN,NaN,NaN,NaN,NaN
093,NaN,NaN,NaN,NaN,NaN,NaN
095,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def produce_analysis_report(path_to_animal_folder, output_folder, animal_number):
    p = pathlib.Path(path_to_animal_folder)
    os.listdir(p)
    folders = [i for i in os.listdir(p) if (p / i).is_dir()]
    ind_first_level = []
    ind_second_level = []
    condition = []
    for folder in sorted(folders):
        if 'block' not in folder:
            date_path = p / folder
            blocks = os.listdir(date_path)
            for block in blocks:
                b_dict = {
                    'automated_step':   False,
                    'manual_sync':      False,
                    'bokeh_plot':       False,
                    'saccade_analysis': False,
                    'accel_analysis':   False
                }
                ind_first_level.append(date_path.name)
                ind_second_level.append(block)
                # get information from analysis folder
                a_files = os.listdir(p / folder / block / 'analysis')
                if 'blocksync_df.csv' in a_files:
                    b_dict['automated_step'] = True
                if len([f for f in a_files if '.html' in f]) != 0:
                    b_dict['bokeh_plot'] = True
                if 'manual_sync_df.csv' in a_files:
                    b_dict['manual_sync'] = True
                if 'l_saccades.csv' in a_files:
                    b_dict['saccade_analysis'] = True



                condition.append(b_dict)

    arrays = [
        np.array(ind_first_level),
        np.array(ind_second_level)
    ]
    df = pd.DataFrame(data=None, index=arrays, columns=['automated_step','manual_synchronization','bokeh_plot', 'saccade_analysis'])
    auto_step_col = [i['automated_step'] for i in condition]
    manual_sync_col = [i['manual_sync'] for i in condition]
    bokeh_plot_col = [i['bokeh_plot'] for i in condition]
    saccade_analysis_col = [i['saccade_anlaysis'] for i in condition]
    df['automated_step'] = auto_step_col
    df['manual_synchronization'] = manual_sync_col
    df['bokeh_plot'] = bokeh_plot_col
    df['saccade_anlaysis'] = saccade_analysis_col
    p = pathlib.Path(output_folder) / f'{animal_number}_analysis_report.csv'
    df.to_csv(p)

In [11]:
output_folder = pathlib.Path(r'Z:\Nimrod\experiments')
animal_folder = pathlib.Path(r'Z:\Nimrod\experiments\PV_24')
produce_analysis_report(animal_folder,output_folder,animal_number='PV24')

In [46]:
p = pathlib.Path(r'\\sil2\data\Nimrod\experiments\PV_24')
folders = os.listdir(p)
ind_first_level = []
ind_second_level = []
condition = []
for folder in sorted(folders):
    if 'block' not in folder:
        date_path = p / folder
        blocks = os.listdir(date_path)
        for block in blocks:
            b_dict = {
                'automated_step': False,
                'manual_sync':    False,
                'bokeh_plot':     False
            }
            ind_first_level.append(date_path.name)
            ind_second_level.append(block)
            a_files = os.listdir(p / folder / block / 'analysis')
            if 'blocksync_df.csv' in a_files:
                b_dict['automated_step'] = True
            if len([f for f in a_files if '.html' in f]) != 0:
                b_dict['bokeh_plot'] = True
            if 'manual_sync_df.csv' in a_files:
                b_dict['manual_sync'] = True
            condition.append(b_dict)

arrays = [
    np.array(ind_first_level),
    np.array(ind_second_level)
]
df = pd.DataFrame(data=None, index=arrays, columns=['automated_step','manual_synchronization','bokeh_plot'])
auto_step_col = [i['automated_step'] for i in condition]
manual_sync_col = [i['manual_sync'] for i in condition]
bokeh_plot_col = [i['bokeh_plot'] for i in condition]
df['automated_step'] = auto_step_col
df['manual_synchronization'] = manual_sync_col
df['bokeh_plot'] = bokeh_plot_col
df

automated_step  manual_synchronization  bokeh_plot
02_01_2022 block_37            True                   False       False
           block_38            True                   False       False
           block_39           False                   False       False
           block_40           False                   False       False
           block_41            True                   False       False
...                             ...                     ...         ...
29_12_2021 block_32           False                   False       False
           block_33           False                   False       False
30_12_2021 block_34           False                   False       False
           block_35            True                   False       False
           block_36           False                   False       False

[92 rows x 3 columns]

In [20]:
arrays= [
    np.array(['02_03_2022','02_03_2022','02_03_2022']),
    np.array(['block_1','block_2','block_2'])
]
df = pd.DataFrame(data=None,index=arrays,columns=['Block'])
df

Block
02_03_2022 block_1   NaN
           block_2   NaN
           block_2   NaN

In [79]:
# reptilearn data sorter draft
p = pathlib.Path(r"D:\experiments\Transients\reptilearn_recordings")
files = os.listdir(p)
for file in files:
        name = file[-19:]
        pre_name = file[:-19]
        format_date = name[6:8] + '_' + name[4:6] + '_' + name[0:4]
        format_time = name[9:11] + '_' + name[11:13] + '_' + name[13:15]
        origin = p / file
        destination = p / format_date
        print(f'would have moved {origin} to {destination}')
        try:
            os.mkdir(destination)
        except FileExistsError:
            print('destination exists')
        destination = p / format_date / format_time
        try:
            os.mkdir(destination)
        except FileExistsError:
            print('destination exists')
        try:
            shutil.move(origin,destination / file)
        except FileExistsError:
            print('this file is already there, moving on...')

would have moved D:\experiments\Transients\reptilearn_recordings\PV24_EE81_back_20220116-130043.csv to D:\experiments\Transients\reptilearn_recordings\16_01_2022
would have moved D:\experiments\Transients\reptilearn_recordings\PV24_EE81_back_20220116-130043.mp4 to D:\experiments\Transients\reptilearn_recordings\16_01_2022
destination exists
destination exists
would have moved D:\experiments\Transients\reptilearn_recordings\PV24_EE81_left_20220116-130043.csv to D:\experiments\Transients\reptilearn_recordings\16_01_2022
destination exists
destination exists
would have moved D:\experiments\Transients\reptilearn_recordings\PV24_EE81_left_20220116-130043.mp4 to D:\experiments\Transients\reptilearn_recordings\16_01_2022
destination exists
destination exists
would have moved D:\experiments\Transients\reptilearn_recordings\PV24_EE81_right_20220116-130043.csv to D:\experiments\Transients\reptilearn_recordings\16_01_2022
destination exists
destination exists
would have moved D:\experiments\Trans

In [17]:
def reptilearn_data_sorter(path_to_dir):
    """
    This function takes a bunch of reptilearn data and sorts them by date and time as per the given format: SomeTitle_CameraName_YYYYMMDD_HHMMSS.mp4 or csv and sorts them as follows:
     date_folder --> Time_folder --> files
    :param path_to_dir: Pathlib.Path object which points to the directory of choice
    :return:
    """
    p = path_to_dir
    files = os.listdir(p)
    for file in files:
        if os.path.isdir(p / file):
            continue
        name = file[-19:]
        format_date = name[6:8] + '_' + name[4:6] + '_' + name[0:4]
        format_time = name[9:11] + '_' + name[11:13] + '_' + name[13:15]
        origin = p / file
        destination = p / format_date
        #print(f'would have moved {origin} to {destination}')
        try:
            os.mkdir(destination)
        except FileExistsError:
            print('destination exists')
        destination = p / format_date / format_time
        try:
            os.mkdir(destination)
        except FileExistsError:
            print('destination exists')
        try:

            shutil.move(origin,destination / file)
        except FileExistsError:
            print('this file is already there, moving on...')

In [18]:
reptilearn_data_sorter(pathlib.Path(r'D:\experiments\Transients\reptilearn_recordings'))

destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination exists
destination 

In [166]:
def reptilearn_organizer_date_to_date(date, path_to_animal_folder, path_to_arena_files):
    """
    This function takes two date folders (with the exact same date format and values) and matches the arena files to OE time signatures - This should be done after all files are organized into date folders on both sides and Open ephys files are in their correct place within the block tree
    :param date: DD_MM_YYYY
    :param path_to_animal_folder: the path to the animal folder (for example, "D:\experiments\PV_24")
    :param path_to_arena_files:  the path to the reptilearn files folder (for example "D:\experiments\Transients\reptilearn_recordings"
    :return: None
    """
    p_dst = pathlib.Path(rf'{path_to_animal_folder}\{date}') #path to a specific date in the animal folder
    p_ori = pathlib.Path(rf'{path_to_arena_files}\{date}') # path to the arena files sorted by date
    dst_ls = os.listdir(p_dst)
    ori_ls = os.listdir(p_ori)
    dst_times = []
    for file in dst_ls:
        p_oe = p_dst / file / 'oe_files'
        filename = os.listdir(p_oe)
        if len(filename) != 1:
            print(f'Missing open ephys file in {file}, cannot work automatically')
            continue
        oe_time = filename[0][-8:]
        t = datetime.timedelta(hours=int(oe_time[0:2]),
                               minutes=int(oe_time[3:5]),
                               seconds=int(oe_time[6:8]))
        dst_times.append([file,t])

    if len(ori_ls) == len(dst_ls):
        print(f'same file count')
        for file in ori_ls:
            arena_time = datetime.timedelta(hours=int(file[0:2]),
                                            minutes=int(file[3:5]),
                                            seconds=int(file[6:8]))
    #        print(f'{file} deltas are')
            deltas = []
            for i in range(len(dst_times)):
                delta = arena_time - dst_times[i][1]
                deltas.append(delta.seconds)
    #            print(f'{dst_times[i][0]} => {delta.seconds}')
            block_position = np.argmin(deltas)
            dst_block = dst_times[block_position][0]
            files_to_move = [p_ori / file / f for f in os.listdir(p_ori / file)]
            print(f'will move {arena_time} to {p_dst / dst_block}')
            print(f'because the delta is {deltas[block_position]}')
            for f in files_to_move:
                shutil.move(f,p_dst / dst_block / "arena_videos" / f.name)
                print(f'moved {f} to {p_dst / dst_block /"arena_videos"/f.name}')
    else:
        print('The file count does not match - work manually you lazy bum')
        print('okay, here are the time differences:')
        for file in ori_ls:
            arena_time = datetime.timedelta(hours=int(file[0:2]),
                                            minutes=int(file[3:5]),
                                            seconds=int(file[6:8]))
    #        print(f'{file} deltas are')
            deltas = []
            print(f'these are the deltas for {file}')
            for i in range(len(dst_times)):
                delta = arena_time - dst_times[i][1]
                deltas.append(delta.seconds)
                print(f'{dst_times[i][0]} => {delta.seconds}')

In [167]:
reptilearn_organizer_date_to_date('04_01_2022', r"D:\experiments\PV_24", r"D:\experiments\Transients\reptilearn_recordings")

Missing open ephys file in block_65, cannot work automatically
same file count
will move 10:41:39 to D:\experiments\PV_24\04_01_2022\block_60
because the delta is 8
moved D:\experiments\Transients\reptilearn_recordings\04_01_2022\10_41_39\back_20220104-104139.csv to D:\experiments\PV_24\04_01_2022\block_60\arena_videos\back_20220104-104139.csv
moved D:\experiments\Transients\reptilearn_recordings\04_01_2022\10_41_39\back_20220104-104139.mp4 to D:\experiments\PV_24\04_01_2022\block_60\arena_videos\back_20220104-104139.mp4
moved D:\experiments\Transients\reptilearn_recordings\04_01_2022\10_41_39\left_20220104-104139.csv to D:\experiments\PV_24\04_01_2022\block_60\arena_videos\left_20220104-104139.csv
moved D:\experiments\Transients\reptilearn_recordings\04_01_2022\10_41_39\left_20220104-104139.mp4 to D:\experiments\PV_24\04_01_2022\block_60\arena_videos\left_20220104-104139.mp4
moved D:\experiments\Transients\reptilearn_recordings\04_01_2022\10_41_39\right_20220104-104139.csv to D:\exper

In [122]:
def build_file_tree(path, blocks_to_build, animal_name_number):
    """
    This function creates the empty file trees required to populate later
    :param path: pathlib.Path location to start the tree
    :param blocks_to_build: an iterable containing block numbers to build
    :param animal_name_number: the name of the folder where the block trees will be created
    """
    #build tree root for specific animal
    try:
        tree_root = pathlib.Path(path / animal_name_number)
        os.mkdir(tree_root)
    except FileExistsError:
        print('Animal folder already exists')
    #create a block template folder
    try:
        block_root = pathlib.Path(tree_root / 'block_template')
        os.mkdir(block_root)
        #make the block template itself:
        os.mkdir(block_root / 'analysis')
        os.mkdir(block_root / 'oe_files')
        os.mkdir(block_root / 'eye_videos')
        os.mkdir(block_root / 'eye_videos' / 'LE')
        os.mkdir(block_root / 'eye_videos' / 'RE')
        os.mkdir(block_root / 'arena_videos')
    except FileExistsError:
        print('template tree already exists')
    #duplicate the template to the number of blocks needed:
    for i in blocks_to_build:
        try:
            folder_to_create = tree_root / f'block_{i}'
            shutil.copytree(block_root,folder_to_create)
            print(f'created {folder_to_create}')
        except FileExistsError:
            print(f'block number {i} already there')

In [179]:
build_file_tree(pathlib.Path(r"D://experiments"),
                range(32,94),
                "PV_24")

Animal folder already exists
template tree already exists
block number 32 already there
block number 33 already there
block number 34 already there
block number 35 already there
block number 36 already there
block number 37 already there
block number 38 already there
block number 39 already there
block number 40 already there
block number 41 already there
block number 42 already there
block number 43 already there
block number 44 already there
block number 45 already there
block number 46 already there
block number 47 already there
block number 48 already there
block number 49 already there
block number 50 already there
block number 51 already there
block number 52 already there
block number 53 already there
block number 54 already there
block number 55 already there
block number 56 already there
block number 57 already there
block number 58 already there
block number 59 already there
block number 60 already there
block number 61 already there
block number 62 already there
block number

In [188]:
# open ephys file mover VERSION1
#initialize
path_to_tree_root = pathlib.Path(r"D://experiments//PV_24")
path_to_oe_data = pathlib.Path(r'\\sil1\Data\Pogona Vitticeps\PV24')
folders_to_move = [i for i in os.listdir(path_to_oe_data) if 'EE' in i]

#move open ephys files
for folder in folders_to_move:
    s = folder[folder.find('EE'):]
    boolean_digit_locations = [b.isdigit() for b in s]
    boolean_underscore_locations = [b == '_' for b in s]
    fd_inds = [int(i+folder.find('EE')) for i,x in enumerate(boolean_digit_locations) if x]
    funder_inds = [int(i+folder.find('EE')) for i,x in enumerate(boolean_underscore_locations) if x]
    if fd_inds[0] < funder_inds[0]:
        block_num = folder[fd_inds[0]:funder_inds[0]]
    else:
        block_num = folder[fd_inds[0]:funder_inds[1]]
    if int(block_num) > 32:
        #print(f'folder {folder} will go to block number {block_num}')
        origin = pathlib.Path(path_to_oe_data / folder)
        destination = pathlib.Path(path_to_tree_root / f'block_{block_num}' / 'oe_files' / folder)
        print(f'Copying {origin} to {destination}')
        try:
            shutil.copytree(origin,destination)
        except FileExistsError:
            print('this file is already there, moving on...')

Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_05_01_22_EE68_2022-01-05_14-40-31 to D:\experiments\PV_24\block_68\oe_files\PV24_05_01_22_EE68_2022-01-05_14-40-31
Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_05_01_22_EE69_2022-01-05_15-12-37 to D:\experiments\PV_24\block_69\oe_files\PV24_05_01_22_EE69_2022-01-05_15-12-37
Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_05_01_22_EE70_2022-01-05_15-48-24 to D:\experiments\PV_24\block_70\oe_files\PV24_05_01_22_EE70_2022-01-05_15-48-24
Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_05_01_22_EE71_2022-01-05_16-20-18 to D:\experiments\PV_24\block_71\oe_files\PV24_05_01_22_EE71_2022-01-05_16-20-18
Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_05_01_22_EE72_2022-01-05_16-44-09 to D:\experiments\PV_24\block_72\oe_files\PV24_05_01_22_EE72_2022-01-05_16-44-09
Copying \\sil1\Data\Pogona Vitticeps\PV24\PV24_10_01_22_EE73_2022-01-10_13-55-31 to D:\experiments\PV_24\block_73\oe_files\PV24_10_01_22_EE73_2022-01-10_13-55-31
Copying \\sil1\Data\Pogona V

In [ ]:
# open ephys file mover VERSION2

In [126]:
# Move eye camera files __VERSION1__
# Initialize:
path_to_tree_root = pathlib.Path(r"D://experiments//PV_24")
eyevids_path_L = pathlib.Path(r"D://experiments//Transients//24//RPI_L_BACKUP_19_01_22")
eyevids_path_R = pathlib.Path(r"D://experiments//Transients//24//RPI_R_BACKUP_19_01_22")
prefixlist = ['LE','RE']
for i, eye in enumerate([eyevids_path_L, eyevids_path_R]):
    folders_to_move = [i for i in os.listdir(eye) if 'EE' in i]
    prefixer = prefixlist[i]
    for s in folders_to_move:
        l = s.find('EE')
        #print(s[l+2:l+5])
        if s[l:l+2] == 'EE_':
            block_num =
        elif s[l+2] == '_':
            #print(f'the block number for {s} is {s[l+3]}')
            block_num = s[l+3]
        elif s[l+3] == '_':
            #print(f'the block number for {s} is {s[l+2]}')
            block_num = s[l+2]
        elif s[l+4] == '_':
            #print(f'the block number for {s} is {s[l+2:l+4]}')
            block_num = s[l+2:l+4]
        origin = pathlib.Path(eye / s)
        destination = pathlib.Path(path_to_tree_root / f'block_{block_num}' / 'eye_videos' / prefixer / s)
        print(f'Copying {origin} to {destination}')
        try:
            #shutil.copytree(origin,destination)
        except FileExistsError:
            print('this file is already there, moving on...')

Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE33_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_33\eye_videos\LE\PV24_EE33_640x480_60hz_experiment_1_recording_0
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE34_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_34\eye_videos\LE\PV24_EE34_640x480_60hz_experiment_1_recording_0
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE35_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_35\eye_videos\LE\PV24_EE35_640x480_60hz_experiment_1_recording_0
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE36_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_36\eye_videos\LE\PV24_EE36_640x480_60hz_experiment_1_recording_0
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE37_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_37\eye_videos\LE\PV24_EE37_640x480_60hz_experiment_1_re

In [180]:
#eye video block identifier + mover Version 2

path_to_tree_root = pathlib.Path(r"D://experiments//PV_24")
eyevids_path_L = pathlib.Path(r"D://experiments//Transients//24//RPI_L_BACKUP_19_01_22")
eyevids_path_R = pathlib.Path(r"D://experiments//Transients//24//RPI_R_BACKUP_19_01_22")
prefixlist = ['LE','RE']
for i, eye in enumerate([eyevids_path_L, eyevids_path_R]):
    folders_to_move = [i for i in os.listdir(eye) if 'EE' in i]
    prefixer = prefixlist[i]
    for folder in folders_to_move:
        s = folder[folder.find('EE'):]
        boolean_digit_locations = [b.isdigit() for b in s]
        boolean_underscore_locations = [b == '_' for b in s]
        fd_inds = [int(i+folder.find('EE')) for i,x in enumerate(boolean_digit_locations) if x]
        funder_inds = [int(i+folder.find('EE')) for i,x in enumerate(boolean_underscore_locations) if x]
        if fd_inds[0] < funder_inds[0]:
            block_num = folder[fd_inds[0]:funder_inds[0]]
        else:
            block_num = folder[fd_inds[0]:funder_inds[1]]
        origin = pathlib.Path(eye / folder)
        destination = pathlib.Path(path_to_tree_root / f'block_{block_num}' / 'eye_videos' / prefixer / folder)
        print(f'Copying {origin} to {destination}')
        try:
            shutil.copytree(origin,destination)
        except FileExistsError:
            print('this file is already there, moving on...')

Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE33_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_33\eye_videos\LE\PV24_EE33_640x480_60hz_experiment_1_recording_0
this file is already there, moving on...
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE34_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_34\eye_videos\LE\PV24_EE34_640x480_60hz_experiment_1_recording_0
this file is already there, moving on...
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE35_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_35\eye_videos\LE\PV24_EE35_640x480_60hz_experiment_1_recording_0
this file is already there, moving on...
Copying D:\experiments\Transients\24\RPI_L_BACKUP_19_01_22\PV24_EE36_640x480_60hz_experiment_1_recording_0 to D:\experiments\PV_24\block_36\eye_videos\LE\PV24_EE36_640x480_60hz_experiment_1_recording_0
this file is already there, moving on...
Copying D:\experiments\Trans

In [84]:
#Dating the blocks:
path_to_tree_root = pathlib.Path(r"D://experiments//PV_24")
block_num = 1
oe_path = pathlib.Path(path_to_tree_root / f'block_{block_num}' / 'oe_files')
s = os.listdir(oe_path)[0]
date_starts = s.find("2021")
date = s[date_starts:date_starts +10]
date


'2021-12-23'

In [71]:
def data_mover_UNFINISHED(path_to_tree_root,path_to_data_source,identifier,data_type):
    """
    This function takes data from a source into the tree by identifier and type
    :param path_to_tree_root:
    :param path_to_data_source:
    :param identifier:
    :param data_type:
    :return:
    """
    #list the directories in the data source
    folders_to_move = [i for i in os.listdir(path_to_data_source) if identifier in i]

    #find the block number
    for s in folders_to_move:
        l = s.find(identifier)
        #print(s[l+2:l+5])
        if s[l+2] == '_':
            #print(f'the block number for {s} is {s[l+3]}')
            block_num = s[l+3]
        elif s[l+3] == '_':
           #print(f'the block number for {s} is {s[l+2]}')
            block_num = s[l+2]
        elif s[l+4] == '_':
            #print(f'the block number for {s} is {s[l+2:l+4]}')
            block_num = s[l+2:l+4]

a
v
s


In [108]:
#cleanup_DLC_files
def cleanup_dlc_files(block_path):
    prefixlist = ['LE','RE']
    for prefixer in prefixlist:
        path = block_path / 'eye_videos' / prefixer
        vid_folder = os.listdir(path)
        path = path / vid_folder[0]
        files_to_save = [path / i for i in os.listdir(path) if 'labeled.mp4' in i]
        for file in files_to_save:
            shutil.move(pathlib.Path(file),path / 'analysis' )
        files_to_remove = [path / i for i in os.listdir(path) if 'DLC' in i]
        for file in files_to_remove:
            os.remove(file)


In [109]:
for i in range(1,6):
    block_path = pathlib.Path(rf'D:\experiments\PV_24\23_12_2021\block_{i}')
    cleanup_dlc_files(block_path)

In [120]:
for i in range(1,6):
    block_path = pathlib.Path(rf'D:\experiments\PV_24\23_12_2021\block_{i}')
    prefixlist = ['LE','RE']
    for prefixer in prefixlist:
       path = block_path / 'eye_videos' / prefixer
       vid_folder = os.listdir(path)
       path = path / vid_folder[0]
       file_name = [i for i in os.listdir(path) if 'oldLabeled.mp4' in i][0]
       file_to_move = path / file_name
       shutil.move(file_to_move, block_path / 'analysis' / file_name)
       #os.rename(path / 'analysis',path / f'{base_name[:-4]}_oldLabeled.mp4')

In [18]:
#### build here: data comparison (check if all data from one computer is present in another for backup purposes) ####
# create a list of data files that exist on computer 1 and either compare them to an existing file or create a file for further comparison:
create_file = True
export_path = pathlib.Path(r'D:\experiments\Transients\file_comparison_for_backup.csv')
p = pathlib.Path(r'D:\experiments\PV_24')
p_ls = list(p.glob('**/oe_files/**'))
name_list = [i.name for i in p_ls if 'PV24' in i.name]
name_dict = {
    'folder_names': name_list
}
if create_file:
    df = pd.DataFrame(name_dict)
    df.to_csv(export_path)
else:
    print('did not export file')

In [14]:
# this cell compares between two similarly formatted .csv files containing folder names and tells you which names are missing from which list
p1 = pathlib.Path(r'D:\experiments\Transients\PV24OEFolderNames_Nimrod.csv')
p2 = pathlib.Path(r'D:\experiments\Transients\PV24OEFolderNames_Maya.csv')
df1 = pd.read_csv(p1)
df2 = pd.read_csv(p2)
for i in df2.folder_names:
    if i not in df1.folder_names.values:
        print(f'{i} missing from {p1}')



OEsetting_PV24 missing from D:\experiments\Transients\PV24OEFolderNames_Nimrod.csv
OEsetting_PV24_2 missing from D:\experiments\Transients\PV24OEFolderNames_Nimrod.csv
PV24_09_03_2022_PC1_2022-03-09_14-43-19 missing from D:\experiments\Transients\PV24OEFolderNames_Nimrod.csv
PV24_11_01_22_Trial_screen1 missing from D:\experiments\Transients\PV24OEFolderNames_Nimrod.csv
PV24_12_1_22_nEE1_2022-01-12_13-56-21 missing from D:\experiments\Transients\PV24OEFolderNames_Nimrod.csv
PV24_12_1_22_nEE2_2022-01-12_14-59-06 missing from D:\experiments\Transients\PV24OEFolderNames_Nimrod.csv
PV24_12_1_22_nEE3_2022-01-12_15-21-29 missing from D:\experiments\Transients\PV24OEFolderNames_Nimrod.csv
PV24_18_1_22_EE84_2022-01-18_16-02-05 missing from D:\experiments\Transients\PV24OEFolderNames_Nimrod.csv
PV24_18_1_22_ElectrodeLowering missing from D:\experiments\Transients\PV24OEFolderNames_Nimrod.csv
PV24_18_1_22_ElectrodeLowering2 missing from D:\experiments\Transients\PV24OEFolderNames_Nimrod.csv
PV24_

In [12]:
'OEsetting_PV24' in df2.folder_names

False

In [32]:
####CAREFUL!!!!####
# This cell deletes all matlab analysis files of a given name
filename = 'lizMov.mat'
# collect date folders:
p = pathlib.Path(r'Z:\Nimrod\experiments\PV_24')
date_folder_list = [i for i in p.iterdir() if 'block' not in str(i).lower() and i.is_dir()]
# for each date, find the block names:
for date_folder in date_folder_list:
    # list all blocks in the folder:
    folder_list = [i for i in date_folder.iterdir()]
    for block in folder_list:
        oe_folder = block / 'oe_files'
        oe_name = [i for i in oe_folder.iterdir()]
        if len(oe_name) == 1:
            try:
                path_to_analysis = oe_name[0] / 'Record Node 101' / 'analysis'
                analysis_folder = [i for i in path_to_analysis.iterdir()][0]
                if filename in [i for i in analysis_folder.iterdir()][0].name:
                    #os.remove(analysis_folder / filename)
                    print(f'would have removed {analysis_folder / filename}')
            except FileNotFoundError:
                print(f'error in {path_to_analysis}')


IndexError: list index out of range